In [35]:
# Import Library
# text preprocessing
import pandas as pd
import numpy as np
import csv
import requests
import io
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re # regular expression
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory # stemming indonesian language
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.corpus import stopwords
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer # to create Bag of words
from sklearn.feature_extraction.text import TfidfVectorizer # tfid Vector 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score # confussion matrix
from sklearn.preprocessing import LabelEncoder # to convert classes to number 
from sklearn.model_selection import train_test_split  # for splitting data 
from sklearn.metrics import accuracy_score # to calculate accuracy
from sklearn.pipeline import Pipeline
#from mlxtend.plotting import plot_confusion_matrix

You should consider upgrading via the 'c:\users\yusrifa deta kirana\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\yusrifa deta kirana\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


[nltk_data] Downloading package punkt to C:\Users\Yusrifa Deta
[nltk_data]     Kirana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Yusrifa Deta
[nltk_data]     Kirana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
dataset = pd.read_csv(r'before_preprocessing.csv')

In [37]:
dataset.head()

,review_id,review_text,price,packaging,product,aroma
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1


In [38]:
dataset.isnull().sum()

review_id      0
review_text    0
price          0
packaging      0
product        1
aroma          0
dtype: int64

In [39]:
dataset = dataset.fillna(method="ffill")
dataset.isnull().sum()

review_id      0
review_text    0
price          0
packaging      0
product        0
aroma          0
dtype: int64

In [40]:
dataset.head()

,review_id,review_text,price,packaging,product,aroma
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1


In [41]:
# casefolding 

def clean(dataset):
  temp_text = []

  for txt in dataset:
    # removal of @name[mention]
    txt = re.sub(r"(?:\@|https?\://)\S+", "", txt)

    # removal of links[https://blabala.com]
    # tw = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", tw)
    txt = re.sub(r"http\S+", "", txt)

    # removal of new line
    txt = re.sub('\n', '', txt)

    # removal of RT
    txt = re.sub('RT', '', txt)

    # removal of punctuations and numbers
    txt = re.sub("[^a-zA-Z^']", " ", txt)
    txt = re.sub(" {2,}", " ", txt)

    # remove leading and trailing whitespace
    txt = txt.strip()

    # remove whitespace with a single space
    txt = re.sub(r'\s+', ' ', txt)

    # convert text to Lowercase
    text = txt.lower();
    temp_text.append(txt)
  return temp_text 

dataset['Clean_text'] = clean(dataset['review_text'])
dataset.head()

,review_id,review_text,price,packaging,product,aroma,Clean_text
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0,sunscreen termahal yang pernah gue beli ini ka...
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0,ini enak banget dipakainya enteng banget diwaj...
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1,the description is quite right produk ini eman...
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0,bisa untuk wajah dan badan dengan harga yang s...
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1,saya beli produk ini karena suka banget wangin...


In [42]:
#TOKENISASI
def token(dataset):
  return dataset.apply(nltk.word_tokenize)

dataset['Clean_text'] = token(dataset['Clean_text'])
dataset.head()

,review_id,review_text,price,packaging,product,aroma,Clean_text
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0,"[sunscreen, termahal, yang, pernah, gue, beli,..."
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0,"[ini, enak, banget, dipakainya, enteng, banget..."
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1,"[the, description, is, quite, right, produk, i..."
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0,"[bisa, untuk, wajah, dan, badan, dengan, harga..."
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1,"[saya, beli, produk, ini, karena, suka, banget..."


In [43]:
#Stopwordremoval

def stop_words(dataset) :
  stop_words = set(stopwords.words('indonesian'))
  return dataset.apply(lambda x: [item for item in x if item not in stop_words])

dataset['Clean_text'] = stop_words(dataset['Clean_text'])
dataset.head()

,review_id,review_text,price,packaging,product,aroma,Clean_text
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0,"[sunscreen, termahal, gue, beli, kayanya, but,..."
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0,"[enak, banget, dipakainya, enteng, banget, diw..."
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1,"[the, description, is, quite, right, produk, e..."
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0,"[wajah, badan, harga, affordable, dipakai, waj..."
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1,"[beli, produk, suka, banget, wanginya, jarang,..."


In [44]:
#Stemming

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem(dataset) :
    hasil = stemmer.stem(dataset)
    return hasil


dataset['Clean_text'] = dataset['Clean_text'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)
dataset.head()

,review_id,review_text,price,packaging,product,aroma,Clean_text
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0,sunscreen termahal gue beli kayanya but it 's ...
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0,enak banget dipakainya enteng banget diwajah l...
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1,the description is quite right produk emang co...
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0,wajah badan harga affordable dipakai wajah oks...
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1,beli produk suka banget wanginya jarang suka p...


In [45]:
dataset['Clean_text'] = dataset.apply(lambda row : stem(row['Clean_text']), axis = 1)

In [46]:
dataset.head()

,review_id,review_text,price,packaging,product,aroma,Clean_text
0,708,sunscreen termahal yang pernah gue beli ini ka...,-1,0,1.0,0,sunscreen mahal gue beli kaya but it s worth i...
1,838,"ini enak banget dipakainya, enteng banget diwa...",0,0,1.0,0,enak banget pakai enteng banget wajah lengket ...
2,1166,the description is quite right. produk ini ema...,0,0,1.0,1,the description is quite right produk emang co...
3,1374,bisa untuk wajah dan badan dengan harga yang s...,1,1,1.0,0,wajah badan harga affordable pakai wajah oksid...
4,1421,saya beli produk ini karena suka banget wangin...,-1,0,0.0,1,beli produk suka banget wangi jarang suka prod...


In [47]:
slang_word = requests.get('https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_slang_words.txt').text
dict_slang = eval(slang_word)

df_acronym = pd.read_csv('https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/acronym.txt', header=None, sep='=')
df_acronym.columns = ['Singkatan', 'kepanjangan']
df_acronym.kepanjangan = df_acronym.kepanjangan.apply(lambda x: x.strip().lower())
df_acronym.Singkatan = df_acronym.Singkatan.apply(lambda x: x.strip().lower())
dict_singkatan = pd.Series(df_acronym.kepanjangan.values,index=df_acronym.Singkatan).to_dict()

dict_clean = {**dict_singkatan, **dict_slang}

In [48]:
def preprocessing_slang_singkatan(review_text, dict_clean=dict_clean):
  regex = r"\b(?:"+"|".join(re.escape(word) for word in dict_clean) + r")\b"
  reobj = re.compile(regex, re.I)
  
  return reobj.sub(lambda x:dict_clean[x.group(0)], review_text)

In [49]:
def clean_all(x, dict_clean=dict_clean):
  x = preprocessing_slang_singkatan(x, dict_clean=dict_clean)
  return x

In [50]:
tqdm.pandas()

In [51]:
dataset['Clean_text'] = dataset.Clean_text.progress_apply(lambda x: clean_all(x, dict_clean=dict_clean))

100%|██████████| 3960/3960 [00:33<00:00, 117.90it/s]


In [52]:
# Select Only Text Column
data1 = dataset[['Clean_text']]
data1

,Clean_text
0,sunscreen mahal saya beli kaya bentuk usaha te...
1,enak banget pakai enteng banget wajah lengket ...
2,the description is quite right produk emang co...
3,wajah badan harga affordable pakai wajah oksid...
4,beli produk suka banget wangi jarang suka prod...
5,kemas oke lotion citra versi universitas kedir...
6,analisis dampak lingkungan kalau rumah tekstur...
7,kali pakai kulit enak banget sudah gitu nggak ...
8,my holygrail suncreen spf tekstur watery moist...
9,tinted sunscreen worth it banget cocok warna t...


In [53]:
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer, CountVectorizer
tfidf_transformer = TfidfTransformer()
tfidf_vectorizer = TfidfVectorizer()
count_vectorize = CountVectorizer()

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'Pusing satu kali',
    'Pusing dua kali',
    'Pusing tiga kali',
    'Pusing lagi']

vectorizer = TfidfVectorizer()

# contoh
a = vectorizer.fit_transform(corpus)
a.toarray()

array([[0.        , 0.49248889, 0.        , 0.40264194, 0.77157901,
        0.        ],
       [0.77157901, 0.49248889, 0.        , 0.40264194, 0.        ,
        0.        ],
       [0.        , 0.49248889, 0.        , 0.40264194, 0.        ,
        0.77157901],
       [0.        , 0.        , 0.88654763, 0.46263733, 0.        ,
        0.        ]])

In [55]:
a = dataset['Clean_text']
# Count TF_IDF Vectorizer
count_vectorizer = CountVectorizer()
count_vector = count_vectorizer.fit_transform(dataset['Clean_text'])
count_vector.shape

(3960, 8588)

In [56]:
a = dataset['Clean_text']
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer = tfidf_vectorizer.fit_transform(a)
print(tfidf_vectorizer)


  (0, 7516)	0.2426534938526257
  (0, 4425)	0.09287693083256633
  (0, 6745)	0.25058733710451886
  (0, 723)	0.07772997152858191
  (0, 3643)	0.12317447081802656
  (0, 761)	0.11332515113690449
  (0, 8194)	0.10371377267789426
  (0, 7809)	0.12947398542523325
  (0, 3360)	0.20155729699374345
  (0, 7227)	0.1355719517148687
  (0, 8467)	0.12947398542523325
  (0, 7118)	0.07807736284086875
  (0, 305)	0.11531360176909397
  (0, 8421)	0.14822494818281082
  (0, 1722)	0.1576054134023332
  (0, 1159)	0.17102937288263842
  (0, 131)	0.18966319337752954
  (0, 7472)	0.07047970458858505
  (0, 7510)	0.211478845660333
  (0, 3037)	0.19792067133543242
  (0, 7328)	0.12174361836327363
  (0, 7870)	0.1772926275041677
  (0, 7891)	0.1360090883720058
  (0, 8401)	0.14573070628999846
  (0, 1223)	0.1584920448042104
  :	:
  (3959, 7019)	0.08557996938285017
  (3959, 4701)	0.1435261862600506
  (3959, 1403)	0.0791127420371088
  (3959, 526)	0.07588226637713823
  (3959, 5355)	0.1250295807605629
  (3959, 3066)	0.17087011901783317


In [57]:
tfidf_vectorizer.shape

(3960, 8588)

In [58]:
x = tfidf_vectorizer.copy()

In [59]:
y = np.array(dataset[['price','packaging','product','aroma']])

In [60]:
from sklearn.preprocessing import MultiLabelBinarizer
Y = MultiLabelBinarizer().fit_transform(y)

In [61]:
from sklearn.model_selection import train_test_split
x_train, x_test, Y_train, Y_test = train_test_split(x,Y, test_size=0.2, random_state=0)

In [91]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

knn2 = KNeighborsClassifier(n_neighbors=2)
knn3 = KNeighborsClassifier(n_neighbors=3)
knn5 = KNeighborsClassifier(n_neighbors=5)
knn7 = KNeighborsClassifier(n_neighbors=7)
knn11 = KNeighborsClassifier(n_neighbors=11)
knn20 = KNeighborsClassifier(n_neighbors=20)
knn25 = KNeighborsClassifier(n_neighbors=25)

## Fit the model on the training data.
classifier2 = MultiOutputClassifier(knn2, n_jobs=-1)
classifier3 = MultiOutputClassifier(knn3, n_jobs=-1)
classifier5 = MultiOutputClassifier(knn5, n_jobs=-1)
classifier7 = MultiOutputClassifier(knn7, n_jobs=-1)
classifier11 = MultiOutputClassifier(knn11, n_jobs=-1)
classifier20 = MultiOutputClassifier(knn20, n_jobs=-1)
classifier25 = MultiOutputClassifier(knn25, n_jobs=-1)

classifier2.fit(x_train, Y_train)
classifier3.fit(x_train, Y_train)
classifier5.fit(x_train, Y_train)
classifier7.fit(x_train, Y_train)
classifier11.fit(x_train, Y_train)
classifier20.fit(x_train, Y_train)
classifier25.fit(x_train, Y_train)

## See how the model performs on the test data.
predictions2 = classifier2.predict(x_test)
predictions3 = classifier3.predict(x_test)
predictions5 = classifier5.predict(x_test)
predictions7 = classifier7.predict(x_test)
predictions11 = classifier11.predict(x_test)
predictions20 = classifier20.predict(x_test)
predictions25 = classifier25.predict(x_test)


In [92]:
knn2_skor=classifier2.score(x_test,np.array(Y_test))
knn3_skor=classifier3.score(x_test,np.array(Y_test))
knn5_skor=classifier5.score(x_test,np.array(Y_test))
knn7_skor=classifier7.score(x_test,np.array(Y_test))
knn11_skor=classifier11.score(x_test,np.array(Y_test))
knn20_skor=classifier20.score(x_test,np.array(Y_test))
knn25_skor=classifier25.score(x_test,np.array(Y_test))

print('knn k-2 :',knn2_skor * 100 ,'%')
print('knn k-3 :',knn3_skor * 100 ,'%')
print('knn k-5 :',knn5_skor * 100 ,'%')
print('knn k-7 :',knn7_skor * 100 ,'%')
print('knn k-11 :',knn11_skor * 100 ,'%')
print('knn k-20 :',knn20_skor * 100 ,'%')
print('knn k-25 :',knn25_skor * 100 ,'%')

knn k-2 : 48.73737373737374 %
knn k-3 : 53.78787878787878 %
knn k-5 : 56.81818181818182 %
knn k-7 : 58.838383838383834 %
knn k-11 : 58.71212121212122 %
knn k-20 : 61.23737373737374 %
knn k-25 : 60.984848484848484 %


In [101]:
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import recall_score
from sklearn.metrics import  multilabel_confusion_matrix

multilabel2 = multilabel_confusion_matrix(Y_test,predictions2)
f1_score2 = metrics.f1_score(Y_test,predictions2, average='micro', labels=np.unique(predictions2))
precision_score2 = metrics.precision_score(Y_test,predictions2, average='micro', labels=np.unique(predictions2))
recall_score2 = metrics.recall_score(Y_test,predictions2, average='micro', labels=np.unique(predictions2))
 
multilabel3 = multilabel_confusion_matrix(Y_test,predictions3)
f1_score3 = metrics.f1_score(Y_test,predictions3, average='micro', labels=np.unique(predictions3))
precision_score3 = metrics.precision_score(Y_test,predictions3, average='micro', labels=np.unique(predictions3))
recall_score3 = metrics.recall_score(Y_test,predictions3, average='micro', labels=np.unique(predictions3))    
    
print('akurasi f1 score        k-2  :', f1_score2* 100 ,'%')
print('akurasi precision score k-2  :', precision_score2* 100 ,'%')
print('akurasi recall score    k-2  :', recall_score2* 100 ,'%')
print('-----------------------------------------------------------')
print('akurasi f1 score        k-3:', f1_score3* 100 ,'%')
print('akurasi precision score k-3:', precision_score3* 100 ,'%')
print('akurasi recall score    k-3:', recall_score3* 100 ,'%')

akurasi f1 score        k-2  : 83.86773547094188 %
akurasi precision score k-2  : 93.31103678929766 %
akurasi recall score    k-2  : 76.16014558689717 %
-----------------------------------------------------------
akurasi f1 score        k-3: 85.71428571428571 %
akurasi precision score k-3: 87.71428571428571 %
akurasi recall score    k-3: 83.80345768880801 %


In [102]:
from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(Y_test,predictions2, target_names=['Class 1', 'Class 2', 'Class 3']))


Classification Report

              precision    recall  f1-score   support

     Class 1       0.58      0.23      0.33       319
     Class 2       0.99      0.98      0.98       780
     Class 3       0.87      0.78      0.82       649

   micro avg       0.91      0.77      0.83      1748
   macro avg       0.82      0.66      0.71      1748
weighted avg       0.87      0.77      0.80      1748
 samples avg       0.92      0.79      0.83      1748

